# HTTP

HTTP是HyperText Transfer Protocol的缩写，翻译后的意思是超文本传输协议。

## http里的时间格式

http里的日期都采用GMT时间格式。
GMT时间格式是符合英语国家的人的习惯，http里的日期时间都采用GMT时间格式，我们随便打开一个网页，通过谷歌浏览器的开发者工具查看Response headers的Date首部，就可以看到类似下面的时间格式

* Data: Mon, 06 Apr 2020 13:20:09 GMT


在http里，允许你使用下面三种时间格式的任意一种
* Mon, 06 Apr 2020 13:20:09 GMT   # RFC 822, updated by RFC 1123
* Monday, 06-Apr-20 13:20:09 GMT  # RFC 850, obsoleted by RFC 1036
* Mon Apr  6 13:20:09 2020   # ANSI C's asctime() format

In [1]:
# UTC  --   GMT
# 当你想生成一个GMT时间时，必须从UTC时间开始进行转换，
# 前两种GMT格式，可以很容易通过strptime方法进行转换
from datetime import datetime

# UTC -> GMT 
# 当前utc转成GMT时间格式
format_1 = '%a, %d %b %Y %H:%M:%S GMT'
format_2 = '%A, %d-%b-%y %H:%M:%S GMT'
utc_now = datetime.utcnow()
gmt_1 = utc_now.strftime(format_1)
gmt_2 = utc_now.strftime(format_2)

print(gmt_1)
print(gmt_2)

# GMT时间格式转成utc   比北京时间慢8h
utc_1 = datetime.strptime(gmt_1, format_1)
utc_2 = datetime.strptime(gmt_2, format_2)
print(utc_1)
print(utc_2)

# 为在http的时间里，使用的时间一定是比我们所用的北京时间慢8个小时，
# 当你处理与http相关的时间时，一定要切记，用utc时间来理解问题和解决问题。

Thu, 11 Apr 2024 16:52:37 GMT
Thursday, 11-Apr-24 16:52:37 GMT
2024-04-11 16:52:37
2024-04-11 16:52:37


In [2]:
# 处理ANSI c 的asctime 时间格式
# 这是第三种可以在http里使用的时间格式，我们可以直接从datetime.utcnow()进行转换，
# 稍麻烦之处在于无法直接使用strftime方法，因为处理日的时间时，
# 小于10的日总是以0开头进行表示，比如3月1日，总是表示成03-01， 
# 这与GMT时间格式不符，但可以使用自定义的字符串格式化方法来解决这个问题
from datetime import datetime

utc_now = datetime.utcnow()
date_str = utc_now.strftime("%a %b") + \
           "  {day}".format(day=utc_now.day) + \
           utc_now.strftime(' %H:%M:%S %Y')


print(date_str)

Thu Apr  11 16:55:28 2024


In [3]:
# 使用 time模块
import time
print(time.asctime(time.gmtime()))

Thu Apr 11 16:56:07 2024


In [5]:
# 如果想把这种格式的字符串转成datetime类型，却可以使用strptime方法
print(datetime.strptime('Mon Apr  6 14:17:40 2020', '%a %b  %d %H:%M:%S %Y'))
# %d 可以将日期6转换为对应的datetime类型中的day， 
# 但是反过来，从datetime数据类型触发，使用%d不能将日期06转换为6。
# 如果你习惯使用time模块来处理时间，那么也可以将这种格式的时间字符串转成struct_time
print(time.strptime('Mon Apr  6 14:17:40 2020', '%a %b  %d %H:%M:%S %Y'))

2020-04-06 14:17:40
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=6, tm_hour=14, tm_min=17, tm_sec=40, tm_wday=0, tm_yday=97, tm_isdst=-1)


# HTTP协议的9种请求方法
* OPTIONS
* GET
* HEAD
* POST
* PUT
* PATCH
* DELETE
* TRACE
* CONNECT

* 频繁使用的GET、POST。

大量使用post请求，区分请求意图的方法是使用不同的uri，比如获取数据用get_xx,新增数据用add_xx， 修改数据用modify_xx，删除数据用del_xx, 一个post请求，实现了对同一个资源的四种操作。